<a href="https://colab.research.google.com/github/yeonsikC/dacon/blob/master/dacon_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\yeonsik\anaconda3\lib\site-packages (20.0.2)


In [0]:
!pip3 install --upgrade tensorflow-gpu

Requirement already up-to-date: tensorflow-gpu in c:\users\yeonsik\anaconda3\lib\site-packages (2.1.0)


In [1]:
from tensorflow.python.client import device_lib 

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12447643684213757134
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16240034726882354889
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7148749528247016645
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14912199066
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1697370349450221405
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [0]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#path = "C:/dacon/temp/"
path="drive/My Drive/Colab Notebooks/datas/dacon/temp/"
train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
sample_submission = pd.read_csv(path+"sample_submission.csv")

In [4]:
train.head()

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
0,0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,0.0,0.0,82.4,71.3,0.0,101.1,0.0,56.1,0.3,0.0,91.0,0.0,1002.5,1009.8,1.1,2.3,0.5,1008.7,12.3,1001.6,69.1,8.2,10.7,1010.1,0.0,256.4,0.0,77.2,62.6,0.0,12.5,11.5,11.0,20.0,20.0,10.5,10.0,10.5,10.0,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,0.0,0.0,81.2,121.8,0.0,165.3,0.0,0.0,0.4,0.0,91.9,0.0,1002.5,1009.9,0.0,0.0,0.0,1008.7,12.1,1001.7,70.3,8.3,10.3,1010.1,0.0,215.4,0.0,77.3,63.5,0.0,12.5,11.0,11.0,20.0,20.0,10.0,9.5,10.0,9.5,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,0.0,0.0,86.1,232.0,0.0,144.0,0.0,0.0,0.6,0.0,93.3,0.0,1002.4,1009.8,0.0,0.0,0.0,1008.6,12.1,1001.6,71.5,8.0,9.7,1010.0,0.0,235.2,0.0,77.3,63.9,0.0,12.0,11.0,11.0,19.5,19.5,10.0,9.0,10.0,9.0,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN
3,3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,0.0,0.0,87.7,0.0,0.0,115.9,0.0,294.1,0.1,0.0,93.4,0.0,1002.4,1009.7,0.0,0.0,0.2,1008.7,12.1,1001.5,73.2,7.7,9.4,1010.1,0.0,214.0,0.0,77.5,64.5,0.0,12.0,11.0,11.0,19.5,19.5,9.5,9.0,9.5,9.0,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,NaN
4,4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,0.0,0.0,88.9,0.0,0.0,124.7,0.0,0.0,0.0,0.0,94.1,0.0,1002.4,1009.7,0.0,0.0,0.0,1008.7,12.0,1001.5,74.3,7.4,9.4,1010.1,0.0,174.9,0.0,78.0,65.0,0.0,12.0,10.5,10.5,19.0,19.5,9.5,8.5,9.5,8.5,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,NaN


In [0]:
Y18_null_train_x=train[train["Y18"].isnull()].iloc[:,1:40]

In [0]:
Y18_null_train_y=train[train["Y18"].isnull()].iloc[:,57]

In [7]:
print(np.array(Y18_null_train_x))

[[9.700e+00 9.888e+02 1.200e+00 ... 0.000e+00 7.720e+01 6.260e+01]
 [9.300e+00 9.889e+02 1.700e+00 ... 0.000e+00 7.730e+01 6.350e+01]
 [9.400e+00 9.890e+02 1.100e+00 ... 0.000e+00 7.730e+01 6.390e+01]
 ...
 [1.950e+01 9.878e+02 9.000e-01 ... 0.000e+00 8.280e+01 7.540e+01]
 [2.000e+01 9.878e+02 1.400e+00 ... 0.000e+00 8.280e+01 7.580e+01]
 [2.010e+01 9.878e+02 1.200e+00 ... 0.000e+00 8.350e+01 7.690e+01]]


In [8]:
print(np.array(Y18_null_train_y).reshape(-1,1))

[[ 8. ]
 [ 7.5]
 [ 7.5]
 ...
 [20. ]
 [20. ]
 [19.5]]


In [0]:
# 0. 사용할 패키지 불러오기
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import np_utils

from sklearn.preprocessing import MinMaxScaler

#데이터 전처리
scaler = MinMaxScaler(feature_range=(0, 1))
Y18_null_train_x = scaler.fit_transform(Y18_null_train_x)

def create_dataset(signal_data_x, signal_data_y, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data_x)-look_back):
        dataX.append(signal_data_x[i:(i+look_back)])
        dataY.append(signal_data_y[i + look_back])
    return np.array(dataX), np.array(dataY)

Y18_null_train_x = np.array(Y18_null_train_x)
Y18_null_train_y = np.array(Y18_null_train_y)

# 랜덤시드 고정시키기
np.random.seed(5)

look_back = 20

x_train, y_train = create_dataset(Y18_null_train_x, Y18_null_train_y, look_back)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 39))

# 1. Y18 = notnull 데이터로 1차 모델 구성
model = Sequential()
model.add(LSTM(128, batch_input_shape = (1, look_back, 39), stateful=True))
#model.add(LSTM(128, batch_input_shape = (1, look_back, 39), stateful=True)) # return_sequences = True 써야함!
#model.add(Dropout(0.2))
model.add(Dense(1))

# 4. 모델 학습과정 설정하기
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
# 전이학습할 때 lr을 줄여야 하므로 optimizer.Adam(lr=0.0001) 처럼 다 고려해야함.
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# lr_reduce, callback 등 해서 점차 lr을 줄이는 방안도 생각해야함.


# 5. 모델 학습시키기
num_epochs = 2000
for epoch_idx in range(num_epochs):
    print ('epochs : ' + str(epoch_idx) )
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2, shuffle=False)
    model.reset_states()

epochs : 0
Epoch 1/1
 - 45s - loss: 66.4684 - mean_squared_error: 66.4684
epochs : 1
Epoch 1/1
 - 44s - loss: 67.2391 - mean_squared_error: 67.2391
epochs : 2
Epoch 1/1
 - 43s - loss: 67.4972 - mean_squared_error: 67.4972
epochs : 3
Epoch 1/1
 - 44s - loss: 62.2675 - mean_squared_error: 62.2675
epochs : 4
Epoch 1/1
 - 43s - loss: 48.2722 - mean_squared_error: 48.2722
epochs : 5
Epoch 1/1
 - 43s - loss: 35.3239 - mean_squared_error: 35.3239
epochs : 6
Epoch 1/1
 - 44s - loss: 19.3368 - mean_squared_error: 19.3368
epochs : 7
Epoch 1/1
 - 43s - loss: 23.5958 - mean_squared_error: 23.5958
epochs : 8
Epoch 1/1
 - 43s - loss: 32.7171 - mean_squared_error: 32.7171
epochs : 9
Epoch 1/1
 - 43s - loss: 17.8792 - mean_squared_error: 17.8792
epochs : 10
Epoch 1/1
 - 43s - loss: 13.3611 - mean_squared_error: 13.3611
epochs : 11
Epoch 1/1
 - 43s - loss: 11.6199 - mean_squared_error: 11.6199
epochs : 12
Epoch 1/1
 - 43s - loss: 11.7142 - mean_squared_error: 11.7142
epochs : 13
Epoch 1/1
 - 44s - loss

In [0]:
 model.save_weight()